In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI()

# import pandas as pd

# df = pd.read_csv('./230001data.csv')

# df.to_csv('230001data.txt', sep="\t")

cache_dir = LocalFileStore('./.cache/')

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\t",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./230001data.txt")
# data = test_csv.load_and_split(text_splitter=splitter)
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages([
  ("system", "다음의 문서에서 질문에 대한 답변을 생성하는 것과 관련이 있는 부분을 카운팅 해주세요 {context}"),
  ("human", "{question}"),
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
  print(inputs)
  documents = inputs['documents']
  question = inputs['question']
  return "\n\n".join(
    map_doc_chain.invoke(
      {"context": doc.page_content, "question": question}
    ).content
    for doc in documents
  )
  

map_chain = {"documents": retriver, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
  ("system", "당신은 스포츠 통계 분석가 입니다. 주어진 문서를 바탕으로 질문에 답하세요. 문서에 없거나 모르는 정보를 지어내지 마세요. 주어진 문서 == {context}"),
  ("human", "{question}")
])


chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("프리그1의 총 경기 숫자를 알려주세요")



{'documents': [Document(page_content='핸디캡\t승3.65\t무3.60\t패1.71\t1 : 0\t승\n\n109\t프리그1\t툴루즈  \tU/O 2.5\t AC아작  \t언더오버\tU1.69\t-\tO1.84\t2\tU\n\n110\t프리그1\t리옹    \t 클레르몽\t일반\t승1.41\t무4.10\t패5.40\t0 : 1\t패\n\n111\t프리그1\t리옹    \tH -1.0\t 클레르몽\t핸디캡\t승2.13\t무3.55\t패2.60\t-1 : 1\t패\n\n112\t프리그1\t리옹    \tH -2.5\t 클레르몽\t핸디캡\t승3.55\t-\t패1.17\t-2.5 : 1\t패\n\n113\t프리그1\t리옹    \tU/O 2.5\t 클레르몽\t언더오버\tU2.19\t-\tO1.47\t1\tU\n\n114\tEPL\t노팅엄포\t 첼시    \t일반\t승5.20\t무3.60\t패1.50\t1 : 1\t무\n\n115\tEPL\t노팅엄포\tH +1.0\t 첼시    \t핸디캡\t승2.21\t무3.35\t패2.60\t2 : 1\t승\n\n116\tEPL\t노팅엄포\tU/O 2.5\t 첼시', metadata={'source': './230001data.txt'}), Document(page_content='무\n\n102\t프리그1\tAS모나코\tH -2.5\t 브레스투\t핸디캡\t승3.46\t-\t패1.18\t-1.5 : 0\t패\n\n103\t프리그1\tAS모나코\tU/O 3.5\t 브레스투\t언더오버\tU1.50\t-\tO2.13\t1\tU\n\n104\t프리그1\t낭트    \t 오세르  \t일반\t승1.80\t무3.35\t패3.55\t1 : 0\t승\n\n105\t프리그1\t낭트    \tH -1.0\t 오세르  \t핸디캡\t승3.30\t무3.40\t패1.85\t0 : 0\t무\n\n106\t프리그1\t낭트    \tU/O 2.5\t 오세르  \t언더오버\tU1.77\t-\tO1.75\t1\tU\n\n107\t프리그1\t

AIMessage(content='프리그1의 총 경기 숫자는 4입니다.')